In [1]:
#import all libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time, sys, requests, random

# import libs, authorize gspread  
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-eaf0e0e671cb.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

In [2]:
#Global
query = ''
numberofPages = 0
cellLocationQuery = 'C2'
cellLocationPages = 'B2'
sheetIndexRead = 0
sheetIndexWrite = 2

#paths/urls
spreadsheet_id = '1vFXonFCyUlEKa1f0s5tvHCKeTek_sAv7rUPYfYss0Qo'
loginurl = 'https://www.linkedin.com/uas/login'
driver_path = '/Users/zacharywong/Documents/Work/Portfolio/DigitalHealthWebscrape/chromedriver'
pathtoLinkedInFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Misc/Companies/Company Web Scrape Tool - Company Names.csv'

#waitTimes
waitLinkedInTime = 10
waitCaptcha = 25
waitSearch= 1
waitLogin = 0.8


In [3]:
def readinValue(cellLocation, sheetIndexRead):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexRead)
    try:
        value = worksheet.acell(cellLocation).value
    except:
        wait()
    return value

In [4]:
def readinQuery():
    global query
    query = readinValue(cellLocationQuery, sheetIndexRead)
    print('query: ' + query)

In [5]:
def readinPages():
    global numberofPages
    numberofPages = int(readinValue(cellLocationPages, sheetIndexRead))
    print('numberofPages: ', numberofPages)

In [6]:
def login(driver):
    driver.get(loginurl)
    email = driver.find_element(By.ID, 'username')
    email.send_keys('zach.jl.wong@gmail.com')
    time.sleep(0.8)
    password = driver.find_element(By.ID, 'password')
    password.send_keys('PandaPanda23')
    time.sleep(0.8)
    password.send_keys(Keys.RETURN)
    

In [7]:
def searchLinkedIn(driver):
    #search query
    try:
        searchBar = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)
    except: 
        time.sleep(waitCaptcha)
        searchBar = driver.find_element(By.XPATH, '//*[@id="global-nav-typeahead"]/input')
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)

In [8]:
def companyFilter(driver):
    WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[2]/button')))
    companyButtons = driver.find_element(By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[2]/button')
    companyButtons.click()

In [9]:
def nextPage(currentPage, driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)
    #WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ember329"]/button')))
    pages = driver.find_elements_by_css_selector("button[type='button']")
    for page in pages:
        #time.sleep(1)
        buttonNumber = page.text
        #print(buttonNumber)
        if buttonNumber == str(currentPage):
            #print('hello')
            page.click()
            break

In [10]:
def addCompanies(driver):
    companies = []
    currentPage = 1
    while (currentPage <= numberofPages):
        time.sleep(2)
        soup = bs(driver.page_source, 'html.parser')
        companySearch = soup.find_all('a', class_ = 'app-aware-link')
        for company in companySearch:
            companyName = company.text
            if '\n\n\n' not in companyName and companyName not in companies:
                companies.append(companyName.replace('\n', ''))
        currentPage += 1
        nextPage(currentPage, driver)
    return companies

In [11]:
def exportCSV(companies):
    df = pd.DataFrame(companies, columns = ['Company Names (Leave Blank if Using LinkedIn Company Mining)'])
    df.to_csv(pathtoLinkedInFile)
    return df

In [12]:
def updateSpreadSheet(df, sheetIndexWrite):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexWrite)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [13]:
def runLinkedIn():
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    readinQuery()
    readinPages()
    login(driver)
    searchLinkedIn(driver)
    companyFilter(driver)
    companies = addCompanies(driver)
    df = exportCSV(companies)
    updateSpreadSheet(df, sheetIndexWrite)
    return df

In [14]:
df = runLinkedIn()
df

query: Digital Health
numberofPages:  4


/var/folders/gl/jgk7f2w528qgjggck6glqp200000gn/T/ipykernel_2810/2957499041.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  pages = driver.find_elements_by_css_selector("button[type='button']")


,Company Names (Leave Blank if Using LinkedIn Company Mining)
0,Digital Health Intelligence Ltd
1,"Proteus Digital Health, Inc"
2,Digital Health & Care Innovation Centre
3,Digital Health Forward
4,Digital Health Coalition
5,Australian Digital Health Agency
6,Digital Health Institute for Transformation (D...
7,Digital Health Canada
8,Digital Health Solutions
9,NHS Digital
